In [2]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

# Devide configuration

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Hyper-parameters

In [4]:
input_size = 28*28
hidden_size = 500
num_classes = 10
num_epochs = 10 
batch_size = 100
learning_rate = 0.001

# MNIST dataset

In [4]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-03-25 06:07:08--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-25 06:07:09--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.1’

MNIST.tar.gz.1          [      <=>           ]  33.20M  5.01MB/s    in 18s     

2021-03-25 06:07:28 (1.80 MB/s) - ‘MNIST.tar.gz.1’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw

In [5]:
train_dataset = torchvision.datasets.MNIST(root='./', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Fully connected neural network with one hidden layer

In [6]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)

    return out

# Create model

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer 

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model

In [9]:
total_step = len(train_loader)  
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Move tensor to the configured device 
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimizer 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
          print ('Epoch [{}/{}], Step [{}/{}], Loss: {}' 
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 2.3016481399536133
Epoch [1/10], Step [200/600], Loss: 2.2456464767456055
Epoch [1/10], Step [300/600], Loss: 2.247933864593506
Epoch [1/10], Step [400/600], Loss: 2.208862066268921
Epoch [1/10], Step [500/600], Loss: 2.199373245239258
Epoch [1/10], Step [600/600], Loss: 2.179702043533325
Epoch [2/10], Step [100/600], Loss: 2.137134552001953
Epoch [2/10], Step [200/600], Loss: 2.1214592456817627
Epoch [2/10], Step [300/600], Loss: 2.0847792625427246
Epoch [2/10], Step [400/600], Loss: 2.011286735534668
Epoch [2/10], Step [500/600], Loss: 2.0570342540740967
Epoch [2/10], Step [600/600], Loss: 1.985325813293457
Epoch [3/10], Step [100/600], Loss: 1.9769103527069092
Epoch [3/10], Step [200/600], Loss: 1.944998025894165
Epoch [3/10], Step [300/600], Loss: 1.8902201652526855
Epoch [3/10], Step [400/600], Loss: 1.8517342805862427
Epoch [3/10], Step [500/600], Loss: 1.8703467845916748
Epoch [3/10], Step [600/600], Loss: 1.8412150144577026
Epoch [4/10], Step

# Test model

In [10]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  
  print('Accuracy: {}'.format(100 * correct / total))

Accuracy: 84.11


# Save model

In [11]:
torch.save(model.state_dict(), 'NN_model.ckpt')